# Section 2.4: Advanced Workflows

| **Aspect** | **Details** |
|-------------|-------------|
| **Goal** | Master prompt chaining patterns for complex workflows. |
| **Time** | ~20 minutes |
| **Prerequisites** | Complete Sections 2.1–2.3 and understand reasoning patterns. |
| **Next Steps** | Continue to Section 2.5: Hands-On Practice |

---

## Quick Setup Check

Since you completed Sections 2.1-2.3, setup is already done. Import it below.

In [ ]:
# Quick setup check - imports setup_utils
try:
    import importlib
    import setup_utils
    importlib.reload(setup_utils)
    from setup_utils import *
    print(f"Setup loaded. Using {AVAILABLE_PROVIDERS} with {get_default_model()}")
    print("Ready to continue.")
except ImportError:
    print("Setup not found.")
    print("Please run 2.1-setup-and-foundations.ipynb first to set up your environment.")

---

### Tactic 6: Prompt Chaining

Break complex tasks into sequential, focused steps.

Complex tasks cause AI to lose focus. Chaining breaks work into steps where each gets full attention.

**Single prompt:** "Review code for security, performance, and style, then fix everything and write tests"
- AI juggles too much at once
- Shallow analysis on each aspect
- Fixes generated without deep understanding

**Chained approach:** Break into focused steps where each builds on the previous

**Core pattern:**
```python
# Step 1
result_1 = ai("First focused task")

# Step 2 uses Step 1 output
result_2 = ai(f"Second task using: {result_1}")

# Step 3 uses Step 2 output  
result_3 = ai(f"Third task using: {result_2}")
```

**When This Matters for Engineers:**

Claude Code, GitHub Copilot, Cursor, and OpenAI Codex use prompt chaining internally for complex tasks. When you ask "review and fix this code," they often chain steps automatically: analyze → fix → verify.

- **Building custom workflows:** Writing agent skills, CI/CD automation, code review bots
- **Debugging tool behavior:** Understanding why Claude Code broke a task into multiple steps
- **Optimizing results:** Manually chaining when automatic chaining isn't sufficient
- **Direct API usage:** Calling OpenAI, Anthropic, or Circuit APIs programmatically

**Industry Standard Patterns:**

Production frameworks like [LangChain](https://docs.langchain.com/oss/python/langchain/overview) formalize these patterns for building AI workflows and agents:

- **Workflows:** Predetermined code paths with defined execution order (what we're teaching here)
- **Agents:** Dynamic systems that choose their own tools and approaches
- **Common patterns:** Sequential chaining, parallelization, routing, orchestrator-worker, evaluator-optimizer

The patterns you'll learn here align with these production frameworks. For implementation details, see [LangGraph workflows documentation](https://docs.langchain.com/oss/python/langgraph/workflows-agents).

**Three common chaining patterns:**

1. **Sequential Workflow:** Linear steps with different focuses (analyze → fix → test)
2. **Self-Correction:** Generate → critique own work → improve (role switching)
3. **Parallel Exploration:** Generate multiple options → evaluate → select best

All three are variations of chaining. Use XML tags (`<analysis>`, `<code>`, `<options>`) to pass data between steps.

#### Pattern 1: Sequential Workflow

Each step has a different focus. Output from one step becomes input to the next.

In [ ]:
# Shared setup helpers (run Section 2.1 first to install dependencies)
from setup_utils import get_chat_completion

In [ ]:
# Vulnerable code example
code = """
def login(username, password):
    query = f"SELECT * FROM users WHERE user='{username}' AND pass='{password}'"
    result = db.execute(query)
    return result
"""

print("=" * 70)
print("STEP 1: Security Analysis")
print("=" * 70)

# Step 1: Focused security analysis
analysis = get_chat_completion([{
    "role": "user",
    "content": f"""Analyze security vulnerabilities in this code:

{code}

List specific issues with severity (CRITICAL/HIGH/MEDIUM/LOW)."""
}])

print(analysis)
print("\n" + "=" * 70)
print("STEP 2: Generate Secure Fix")
print("=" * 70)

# Step 2: Fix based on analysis
fix = get_chat_completion([{
    "role": "user",
    "content": f"""Security issues found:
{analysis}

Original code:
{code}

Provide fixed code with:
- Parameterized queries
- Input validation  
- Error handling

Wrap code in <code></code> tags."""
}])

print(fix)

print("\n💡 Each step focused on one thing. Step 2 had full context from Step 1.")

#### Pattern 2: Self-Correction

AI generates, then critiques its own work, then improves it. Role switching from creator to critic.

```
Generate → Critique own work → Fix issues
```

This combines chaining with role switching and weighted criteria.

In [ ]:
# Helper functions
def extract_between_tags(text, tag):
    start = f"<{tag}>"
    end = f"</{tag}>"
    if start in text and end in text:
        return text[text.find(start) + len(start):text.find(end)].strip()
    return text

# Step 1: Generate
print("Step 1: Generate password validator...")
draft = get_chat_completion([{"role": "user", "content": "Create Python function to validate password strength. Wrap in <code> tags."}])
code = extract_between_tags(draft, "code")
print(f"Generated {len(code)} characters of code")
print(code)
print()

# Step 2: Critique with weighted criteria
print("Step 2: AI reviews its own code (role switch to critic)...")
critique = get_chat_completion([{"role": "system", "content": "You are a security-focused code reviewer."},
    {"role": "user", "content": f"""Review this code:
{code}

Weighted scoring:
- Security (50%): Weak password risks, timing attacks
- Validation (30%): Length, complexity checks  
- Usability (20%): Error messages, feedback

For each:
- Score 0-10
- List issues
- Severity: CRITICAL/HIGH/MEDIUM/LOW

Format: <issues>[list]</issues>"""}])

issues = extract_between_tags(critique, "issues")
print(f"Issues found:")
print(issues)
print()

# Step 3: Self-improve
print("Step 3: AI fixes identified issues...")
improved = get_chat_completion([{"role": "user", "content": f"""Original:
{code}

Issues:
{issues}

Fix all CRITICAL and HIGH issues. Keep changes minimal. Wrap in <improved> tags."""}])
final_code = extract_between_tags(improved, "improved")
print(f"✅ Improved version ({len(final_code)} characters)")
print(final_code)

print("\n💡 Self-correction catches mistakes automatically before human review.")

#### Key Takeaways: Prompt Chaining

**Best Practices Demonstrated:**
1. **One focus per step:** Each prompt has a single, clear objective
2. **Sequential dependencies:** Later steps use earlier outputs as input
3. **XML structure:** Tags like `<analysis>`, `<code>`, `<issues>` pass data cleanly
4. **Role switching:** AI can act as creator (Step 1) then critic (Step 2)
5. **Weighted criteria:** Prioritize what matters (security bugs > missing docs)

**Common chaining patterns:**
- **Sequential workflow:** Step 1 → Step 2 → Step 3 (each builds on previous)
- **Self-improvement:** Generate → Critique → Fix (role switch between steps)
- **Iterative refinement:** Generate → Critique → Fix → Verify → Fix again

**When to use:**
- Multi-step workflows (analyze → fix → test)
- Quality-critical tasks (code reviews, security audits)
- Complex refactoring with verification steps

**When NOT to use:**
- Simple, single-step tasks
- Speed matters more than quality
- No intermediate verification needed

### 🎯 Try It Yourself: Prompt Chaining

**Common Misconception:** AI can handle multiple complex tasks in one prompt as well as breaking them into steps.

**The Reality:** Chaining gives each task full attention, dramatically improving quality.

**Your Task:** Compare single prompt vs chained approach. First run the BAD example, then uncomment the GOOD chained version.

In [ ]:
vulnerable_code = """
def process_payment(amount, card):
    if amount > 0:
        charge(card, amount)
        return "success"
"""

# ❌ BAD: Everything at once
print("Single Prompt: Do everything at once")
single = get_chat_completion([{"role": "user", "content": f"Review, fix, and test this code:\n{vulnerable_code}"}])
print(single)
print()

# ✅ YOUR TURN: Uncomment to try chained approach
# print("\nChained: Three focused steps")
# 
# # Step 1: Just analyze
# analysis = get_chat_completion([{"role": "user", "content": f"List security and validation issues:\n{vulnerable_code}"}])
# print(f"Step 1 - Analysis:\n{analysis}\n")
# 
# # Step 2: Just fix
# fixed = get_chat_completion([{"role": "user", "content": f"Fix issues:\n{analysis}\n\nCode:\n{vulnerable_code}"}])
# print(f"Step 2 - Fixed Code:\n{fixed}\n")
# 
# # Step 3: Just test
# tests = get_chat_completion([{"role": "user", "content": f"Write tests for:\n{fixed}"}])
# print(f"Step 3 - Tests:\n{tests}")

---

#### Pattern 3: Parallel Exploration

Generate multiple approaches simultaneously, then evaluate and pick the best one.

**The idea:** Instead of asking for one approach, generate several at the same time. Compare them, then choose.

**Why parallel?**
- **Sequential:** Ask for approach A → wait → ask for approach B → wait → ask for approach C (slow)
- **Parallel:** Ask for A, B, and C all at once → wait once → get all three (fast)

You'll see this in action below. Parallel execution typically runs 3-5x faster.

**When this helps:**
- Choosing between algorithms or data structures
- Picking libraries or frameworks
- Architecture decisions where multiple options exist
- Any time you need to compare trade-offs

**Skip it when:**
- There's a clear best practice already
- You're rate-limited by the API
- Speed isn't important

**How it works:**

<div style="margin: 20px 0; padding: 20px; background: #f8fafc; border-radius: 8px; font-family: monospace; font-size: 13px;">
  <div style="text-align: center; margin-bottom: 15px;">
    <div style="background: #fee2e2; padding: 8px 16px; border-radius: 6px; display: inline-block; color: #991b1b; font-weight: 600;">User Query</div>
  </div>
  <div style="text-align: center; margin-bottom: 15px;">
    <div style="font-size: 20px; color: #64748b;">↓</div>
  </div>
  <div style="text-align: center; margin-bottom: 15px;">
    <div style="background: #e0e7ff; padding: 12px 20px; border-radius: 6px; display: inline-block; color: #3730a3; font-weight: 600;">Generate 3 approaches simultaneously</div>
  </div>
  <div style="text-align: center; margin-bottom: 15px;">
    <div style="font-size: 20px; color: #64748b;">↓</div>
  </div>
  <div style="display: flex; justify-content: center; gap: 15px; margin-bottom: 15px; flex-wrap: wrap;">
    <div style="background: #dcfce7; padding: 10px 16px; border-radius: 6px; color: #166534; font-weight: 600; min-width: 100px; text-align: center;">Option A</div>
    <div style="background: #dcfce7; padding: 10px 16px; border-radius: 6px; color: #166534; font-weight: 600; min-width: 100px; text-align: center;">Option B</div>
    <div style="background: #dcfce7; padding: 10px 16px; border-radius: 6px; color: #166534; font-weight: 600; min-width: 100px; text-align: center;">Option C</div>
  </div>
  <div style="text-align: center; margin-bottom: 15px;">
    <div style="font-size: 20px; color: #64748b;">↓</div>
  </div>
  <div style="text-align: center; margin-bottom: 15px;">
    <div style="background: #fef3c7; padding: 12px 20px; border-radius: 6px; display: inline-block; color: #92400e; font-weight: 600;">Evaluate all approaches</div>
  </div>
  <div style="text-align: center; margin-bottom: 15px;">
    <div style="font-size: 20px; color: #64748b;">↓</div>
  </div>
  <div style="text-align: center;">
    <div style="background: #dbeafe; padding: 12px 20px; border-radius: 6px; display: inline-block; color: #1e40af; font-weight: 600;">Select best with reason</div>
  </div>
</div>

**Example use case:** Algorithm selection, architecture decisions, library comparisons

In [ ]:
import asyncio
import time
from setup_utils import get_chat_completion_async, run_async

problem = "Find duplicates in 1 million records"

async def generate_approach(approach_name, problem):
    messages = [{"role": "user", "content": f"""{problem}

Generate approach: {approach_name}
Provide: Name, algorithm, time/space complexity"""}]
    return await get_chat_completion_async(messages)

print("=" * 70)
print("SEQUENTIAL vs PARALLEL")
print("=" * 70)

# Sequential: one at a time
print("\nSequential (one at a time):")
print("-" * 70)
start_seq = time.time()

approach_a = get_chat_completion([{"role": "user", "content": f"""{problem}
Generate approach: Hash Set
Provide: Name, algorithm, time/space complexity"""}])
print(f"✓ Hash Set ({time.time() - start_seq:.1f}s)")

approach_b = get_chat_completion([{"role": "user", "content": f"""{problem}
Generate approach: Sorting
Provide: Name, algorithm, time/space complexity"""}])
print(f"✓ Sorting ({time.time() - start_seq:.1f}s)")

approach_c = get_chat_completion([{"role": "user", "content": f"""{problem}
Generate approach: Nested Loops
Provide: Name, algorithm, time/space complexity"""}])
print(f"✓ Nested Loops ({time.time() - start_seq:.1f}s)")

sequential_time = time.time() - start_seq
print(f"\nTotal: {sequential_time:.1f} seconds")

# Parallel: all at once
print("\nParallel (all at once):")
print("-" * 70)

async def generate_all_parallel():
    start = time.time()
    results = await asyncio.gather(
        generate_approach("Hash Set", problem),
        generate_approach("Sorting", problem),
        generate_approach("Nested Loops", problem)
    )
    elapsed = time.time() - start
    return results, elapsed

results, parallel_time = run_async(generate_all_parallel())

approach_a_par, approach_b_par, approach_c_par = results

print(f"✓ All 3 generated in parallel")
print(f"Total: {parallel_time:.1f} seconds")
print(f"Speedup: {sequential_time / parallel_time:.1f}x faster\n")

# Step 2: Evaluate all approaches
print("=" * 70)
print("Step 2: Evaluate approaches")
print("=" * 70)

alternatives = f"""Hash Set: {approach_a_par}

Sorting: {approach_b_par}

Nested Loops: {approach_c_par}"""

evaluation = get_chat_completion([{"role": "user", "content": f"""Compare these approaches for finding duplicates in 1M records:

{alternatives}

Score each 1-10 on:
- Performance (time complexity)
- Memory usage
- Scalability
- Code simplicity

Format as a comparison table."""}])
print(evaluation)

# Step 3: Pick the winner
print("\n" + "=" * 70)
print("Step 3: Select best approach")
print("=" * 70)

selection = get_chat_completion([{"role": "user", "content": f"""Based on this evaluation:

{evaluation}

Which approach is best for 1M records? Explain why."""}])
print(selection)

print("\n" + "=" * 70)
print("Takeaway")
print("=" * 70)
print(f"Parallel execution saved {sequential_time - parallel_time:.1f} seconds.")
print("Generated 3 options → Compared them → Picked the best.")

---

### 🎯 Try It Yourself: Prompt Chaining Patterns

**Common Misconception:** AI can handle complex decisions in one prompt as well as breaking them into focused steps.

**The Reality:** Chaining patterns give each task full attention, dramatically improving quality.

**Your Task:** Try the parallel exploration pattern. Compare single-path vs exploring multiple alternatives.

**Before you start, think about:**

Could you please share:

- What options are you choosing between?

- What type of system/solution is this for?

- What are your priorities (performance vs. cost vs. simplicity)?

**Example answers:**

For the caching example below:
- **Options:** Client-side (browser cache), CDN (edge caching), Server-side (Redis/Memcached)
- **System type:** API caching strategies to reduce database load
- **Priorities:** Latency reduction, DB load reduction, Cost, Complexity, Cache invalidation ease

Use these answers to customize the example code below with your specific problem and options.

In [ ]:
# ❌ BAD: Single path (first idea that comes to mind)
print("Single Path Approach:")
single = get_chat_completion([{"role": "user", "content": "Design caching for API to reduce DB load"}])
print(single)
print()

# ✅ YOUR TURN: Uncomment for parallel decision support approach
# import asyncio
# from setup_utils import get_chat_completion_async, run_async

# # Customize these values based on your specific decision:
# # - problem: What type of system/solution is this for?
# #   Example: "API caching strategies to reduce DB load"
# # - option names: What options are you choosing between?
# #   Example: ["Client-side (browser cache)", "CDN (edge caching)", "Server-side (Redis/Memcached)"]
# # - evaluation criteria: What are your priorities (performance vs. cost vs. simplicity)?
# #   Example: Latency reduction, DB load reduction, Cost, Complexity, Cache invalidation ease
# problem = "API caching strategies to reduce DB load"

# async def generate_option(option_name, problem):
#     """Generate a single caching option asynchronously."""
#     messages = [{"role": "user", "content": f"""You are a systems architect designing caching solutions.

# Context: {problem}

# Generate a detailed approach for: {option_name}

# Provide a comprehensive explanation covering:

# 1. **How it works**: Explain the mechanism and architecture
# 2. **What it caches**: Specify what data/content is cached
# 3. **TTL (Time To Live)**: Recommended cache expiration strategy
# 4. **Implementation details**: 
#    - Required infrastructure/components
#    - Code examples or configuration snippets
#    - Integration points
#    - Key considerations or limitations

# Format your response clearly with section headers for each part."""}]
#     return await get_chat_completion_async(messages)

# print("=" * 70)
# print("Decision Support Approach: Parallel Exploration")
# print("=" * 70)

# # Step 1: Generate all options in parallel
# # This step uses async/await to generate multiple options simultaneously.
# # Instead of waiting for each option one-by-one, asyncio.gather() runs all three
# # API calls concurrently, dramatically reducing total wait time.
# print("\nStep 1: Generate all options in parallel")
# print("-" * 70)

# async def generate_all_options():
#     # asyncio.gather() runs all three async calls concurrently
#     # Returns results in the same order as the input arguments
#     results = await asyncio.gather(
#         generate_option("Client-side (browser cache)", problem),
#         generate_option("CDN (edge caching)", problem),
#         generate_option("Server-side (Redis/Memcached)", problem)
#     )
#     return results

# results = run_async(generate_all_options())
# client_side, cdn, server_side = results

# print("✓ Client-side option generated")
# print("✓ CDN option generated")
# print("✓ Server-side option generated\n")

# # Step 2: Evaluate all approaches
# # Now that we have all options, we compare them using structured criteria.
# # This step is sequential (not parallel) because it depends on Step 1's output.
# # The AI evaluates each option against the same criteria for fair comparison.
# print("=" * 70)
# print("Step 2: Evaluate all approaches")
# print("=" * 70)

# alternatives = f"""Client-side: {client_side}

# CDN: {cdn}

# Server-side: {server_side}"""

# scores = get_chat_completion([
#     {"role": "system", "content": "You are an expert systems architect. You MUST format your evaluations as markdown tables with numeric scores."},
#     {"role": "user", "content": f"""Evaluate these caching options and provide scores in a comparison table.

# <context>
# {problem}
# </context>

# <options_to_evaluate>
# {alternatives}
# </options_to_evaluate>

# <evaluation_criteria>
# Score each option from 1-10 (where 10 is best) on:
# 1. Latency Reduction - How much does this reduce response time?
# 2. DB Load Reduction - How much does this reduce database queries?
# 3. Cost - Lower cost = higher score (infrastructure, maintenance, etc.)
# 4. Complexity - Lower complexity = higher score (implementation and maintenance)
# 5. Cache Invalidation Ease - How easy is it to invalidate/update cached data?

# Overall Score = sum of all 5 criteria (max 50)
# </evaluation_criteria>

# <output_format>
# You MUST respond with a markdown table. Here is the EXACT format required:

# ## Comparison Table

# | Option | Latency Reduction | DB Load Reduction | Cost | Complexity | Cache Invalidation Ease | Overall Score |
# |--------|-------------------|-------------------|------|------------|------------------------|---------------|
# | Client-side | 8/10 | 3/10 | 9/10 | 7/10 | 4/10 | 31/50 |
# | CDN | 9/10 | 7/10 | 5/10 | 6/10 | 5/10 | 32/50 |
# | Server-side | 6/10 | 9/10 | 7/10 | 6/10 | 8/10 | 36/50 |

# ## Key Trade-offs

# [Your analysis here]
# </output_format>

# Now evaluate the three options above and provide your scores in the table format shown. Replace the example scores with your actual evaluation."""}
# ])
# print(scores)

# # Step 3: Recommend best option
# # Final step uses the evaluation scores to make a decision.
# # This is sequential because it needs the scores from Step 2.
# # The AI synthesizes the evaluation data to recommend the best option or hybrid approach.
# # IMPORTANT: Include all context (problem, options, criteria, scores) so AI has full picture.
# print("\n" + "=" * 70)
# print("Step 3: Select best approach")
# print("=" * 70)

# best = get_chat_completion([{"role": "user", "content": f"""Context: {problem}

# Options evaluated:
# {alternatives}

# Evaluation results:
# {scores}

# Based on the evaluation above, recommend the best option (or hybrid approach) for this use case.
# Justify your choice by referencing specific scores and trade-offs from the evaluation."""}])
# print(best)

# print("\n" + "=" * 70)
# print("Takeaway")
# print("=" * 70)
# print("Generated 3 options in parallel → Compared them → Picked the best.")

---

## Summary

**Prompt chaining: One technique, three patterns**

| Pattern | Use Case | Steps | Cost |
|---------|----------|-------|------|
| **Sequential Workflow** | Linear multi-step tasks | Step 1 → Step 2 → Step 3 (different focuses) | $$ |
| **Self-Correction** | Quality improvement | Generate → Critique → Improve (role switch) | $$ |
| **Parallel Exploration** | Choosing between options | Generate N → Evaluate → Select best | $$$ |

**Best Practices:**
1. **One focus per step:** Each prompt has a single, clear objective
2. **Sequential dependencies:** Later steps use earlier outputs as input
3. **XML structure:** Tags like `<analysis>`, `<code>`, `<options>` pass data cleanly
4. **Role switching:** AI can act as creator then critic (self-correction)
5. **Weighted criteria:** Prioritize what matters (security > style)

**When to use chaining:**
- Multi-step workflows (analyze → fix → test)
- Quality-critical tasks (code reviews, security audits)
- Complex decisions with multiple viable approaches
- Need to verify each intermediate stage

**When NOT to use:**
- Simple, single-step tasks
- Speed matters more than quality
- No intermediate verification needed
- Clear best practices already exist

**Cost reality:**
- Single prompt: 1 call
- Sequential/Self-correction: 2-3 calls
- Parallel exploration: 3-7 calls (N alternatives + evaluation + selection)

More calls = better quality. Use strategically for important decisions and complex tasks.

---

<div style="margin:24px 0; padding:20px 24px; background:linear-gradient(135deg, #f8fafc 0%, #e2e8f0 100%); border-radius:12px; border-left:5px solid #10b981; box-shadow:0 2px 8px rgba(0,0,0,0.1);">
  <div style="color:#1e293b; font-size:0.85em; font-weight:600; text-transform:uppercase; letter-spacing:1px; margin-bottom:8px;">⏭️ Next Section</div>
  <div style="color:#0f172a; font-size:1.15em; font-weight:700; margin-bottom:6px;">Section 2.5: Hands-On Practice</div>
  <div style="color:#475569; font-size:0.95em; line-height:1.5; margin-bottom:12px;">Apply all tactics independently in unguided practice activities with automated evaluation.</div>
  <a href="./2.5-hands-on-practice.ipynb" style="display:inline-block; padding:8px 16px; background:#10b981; color:#fff; text-decoration:none; border-radius:6px; font-weight:600; font-size:0.9em; transition:all 0.2s;">Continue to Section 2.5 →</a>
</div>